# CONFLUENCE Tutorial: Distributed Basin Workflow with Delineation

This notebook demonstrates the distributed modeling approach using the delineation method. We'll use the same Bow River at Banff location but create a distributed model with multiple GRUs (Grouped Response Units).

## Key Differences from Lumped Model

- **Domain Method**: `delineate` instead of `lumped`
- **Stream Threshold**: 5000 (creates more sub-basins)
- **Multiple GRUs**: Each sub-basin becomes a GRU
- **Routing**: mizuRoute connects the GRUs
- **Spatial Mode**: Distributed instead of Lumped

## Learning Objectives

1. Understand watershed delineation with stream networks
2. Create a distributed model with multiple GRUs
3. Compare with lumped approach from Tutorial 1
4. Learn how to reuse data across experiments

## 1. Setup and Import Libraries

In [ ]:
# Import required libraries
import sys
import os
from pathlib import Path
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from datetime import datetime
import numpy as np
import contextily as cx
import xarray as xr
from IPython.display import Image, display

# Add CONFLUENCE to path
confluence_path = Path('../').resolve()
sys.path.append(str(confluence_path))

# Import main CONFLUENCE class
from CONFLUENCE import CONFLUENCE

# Set up plotting style
plt.style.use('default')
%matplotlib inline

## 2. Initialize CONFLUENCE
First, let's set up our directories and load the configuration. We'll modify the configuration from Tutorial 1 to create a distributed model.

In [ ]:
# Set directory paths
CONFLUENCE_CODE_DIR = confluence_path
CONFLUENCE_DATA_DIR = Path('/work/comphyd_lab/data/CONFLUENCE_data')  # ← User should modify this path

# Verify paths exist
if not CONFLUENCE_CODE_DIR.exists():
    raise FileNotFoundError(f"CONFLUENCE code directory not found: {CONFLUENCE_CODE_DIR}")

if not CONFLUENCE_DATA_DIR.exists():
    print(f"Data directory doesn't exist. Creating: {CONFLUENCE_DATA_DIR}")
    CONFLUENCE_DATA_DIR.mkdir(parents=True, exist_ok=True)

# Load template configuration
config_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_template.yaml'

# Read config file
with open(config_path, 'r') as f:
    config_dict = yaml.safe_load(f)

# Update core paths
config_dict['CONFLUENCE_CODE_DIR'] = str(CONFLUENCE_CODE_DIR)
config_dict['CONFLUENCE_DATA_DIR'] = str(CONFLUENCE_DATA_DIR)

# Modify for distributed delineation
config_dict['DOMAIN_NAME'] = 'Bow_at_Banff_distributed'
config_dict['EXPERIMENT_ID'] = 'distributed_tutorial'
config_dict['DOMAIN_DEFINITION_METHOD'] = 'delineate'  # Changed from 'lumped'
config_dict['STREAM_THRESHOLD'] = 5000  # Higher threshold for fewer sub-basins
config_dict['DOMAIN_DISCRETIZATION'] = 'GRUs'  # Keep as GRUs
config_dict['SPATIAL_MODE'] = 'Distributed'  # Changed from 'Lumped'

# Use just one parallel process for this tutorial
config_dict['MPI_PROCESSES'] = 1

# Save updated config to a temporary file
temp_config_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_distributed.yaml'
with open(temp_config_path, 'w') as f:
    yaml.dump(config_dict, f)

# Initialize CONFLUENCE
confluence = CONFLUENCE(temp_config_path)

# Display configuration
print("=== Directory Configuration ===")
print(f"Code Directory: {CONFLUENCE_CODE_DIR}")
print(f"Data Directory: {CONFLUENCE_DATA_DIR}")
print("\n=== Key Configuration Settings ===")
print(f"Domain Name: {confluence.config['DOMAIN_NAME']}")
print(f"Pour Point: {confluence.config['POUR_POINT_COORDS']}")
print(f"Domain Method: {confluence.config['DOMAIN_DEFINITION_METHOD']}")
print(f"Stream Threshold: {confluence.config['STREAM_THRESHOLD']}")
print(f"Spatial Mode: {confluence.config['SPATIAL_MODE']}")
print(f"Model: {confluence.config['HYDROLOGICAL_MODEL']}")
print(f"Simulation Period: {confluence.config['EXPERIMENT_TIME_START']} to {confluence.config['EXPERIMENT_TIME_END']}")

## 3. Visualize Lumped vs Distributed Concept

Before we start building the distributed model, let's visualize the difference between lumped and distributed approaches.

In [ ]:
# Create visualization comparing lumped vs distributed
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Lumped representation
ax1.add_patch(plt.Rectangle((0, 0), 1, 1, fill=True, color='lightblue', alpha=0.7))
ax1.text(0.5, 0.5, 'Entire Basin\n(1 Unit)', ha='center', va='center', fontsize=14, fontweight='bold')
ax1.arrow(0.5, 0.05, 0, -0.1, head_width=0.03, head_length=0.02, fc='blue', ec='blue')
ax1.text(0.5, -0.08, 'Single Output', ha='center', fontsize=10)
ax1.set_xlim(-0.1, 1.1)
ax1.set_ylim(-0.15, 1.1)
ax1.set_title('Lumped Model (Tutorial 1)', fontsize=16, fontweight='bold')
ax1.axis('off')

# Distributed representation
# Create sub-basins with river network
colors = ['lightblue', 'lightgreen', 'lightcoral', 'lightyellow']
positions = [(0.2, 0.7), (0.7, 0.7), (0.2, 0.3), (0.5, 0.1)]
labels = ['GRU 1', 'GRU 2', 'GRU 3', 'GRU 4']

for i, (pos, color, label) in enumerate(zip(positions, colors, labels)):
    circle = plt.Circle(pos, 0.15, fill=True, color=color, alpha=0.7, edgecolor='black')
    ax2.add_patch(circle)
    ax2.text(pos[0], pos[1], label, ha='center', va='center', fontsize=10, fontweight='bold')

# Draw river network
ax2.plot([0.2, 0.2, 0.5], [0.7, 0.3, 0.1], 'b-', linewidth=3)  # Main stem
ax2.plot([0.7, 0.5], [0.7, 0.1], 'b-', linewidth=2)  # Tributary
ax2.arrow(0.5, 0.1, 0, -0.1, head_width=0.03, head_length=0.02, fc='blue', ec='blue')
ax2.text(0.5, -0.02, 'Routed Output', ha='center', fontsize=10)

ax2.set_xlim(-0.1, 1.1)
ax2.set_ylim(-0.15, 1.1)
ax2.set_title('Distributed Model (This Tutorial)', fontsize=16, fontweight='bold')
ax2.axis('off')

plt.tight_layout()
plt.show()

## 4. Project Setup - Organizing the Modeling Workflow

First, we'll establish a well-organized project structure, similar to what we did in Tutorial 1.

In [ ]:
# Step 1: Project Initialization
print("=== Step 1: Project Initialization ===")

# Setup project
project_dir = confluence.managers['project'].setup_project()

# Create pour point
pour_point_path = confluence.managers['project'].create_pour_point()

# List created directories
print("\nCreated directories:")
for item in sorted(project_dir.iterdir()):
    if item.is_dir():
        print(f"  📁 {item.name}")

print("\nNote: The pour point location is identical to the lumped model.")
print("The difference is in how we subdivide the watershed above this point.")

## 5. Geospatial Domain Definition - Data Acquisition and Preparation

We'll reuse some of the geospatial data from the lumped model tutorial, where appropriate.

In [ ]:
# Check if we can reuse data from the lumped model
lumped_dem_path = CONFLUENCE_DATA_DIR / 'domain_Bow_at_Banff_lumped' / 'attributes' / 'elevation' / 'dem'
can_reuse = lumped_dem_path.exists()

if can_reuse:
    print("Found existing geospatial data from lumped model. This will speed up the process.")
    print("The distributed model can reuse some of these data layers.")
else:
    print("No existing data found from the lumped model. Will acquire all data from scratch.")

# Step 2: Geospatial Domain Definition - Data Acquisition
print("\n=== Step 2: Geospatial Domain Definition - Data Acquisition ===")

# Acquire attributes
print("Acquiring geospatial attributes (DEM, soil, land cover)...")
confluence.managers['data'].acquire_attributes()

print("\n✓ Geospatial attributes acquired")

## 6. Geospatial Domain Definition - Delineation with Stream Network

This is where the main difference occurs - we'll create multiple sub-basins connected by a stream network.

In [ ]:
# Step 3: Geospatial Domain Definition - Delineation
print("=== Step 3: Geospatial Domain Definition - Delineation ===")

# Define domain
print(f"Delineating distributed watershed...")
print(f"Method: {confluence.config['DOMAIN_DEFINITION_METHOD']}")
print(f"Stream threshold: {confluence.config['STREAM_THRESHOLD']}")
print("\nThis will create multiple sub-basins connected by a stream network.")

watershed_path = confluence.managers['domain'].define_domain()

# Check outputs
basin_path = project_dir / 'shapefiles' / 'river_basins'
network_path = project_dir / 'shapefiles' / 'river_network'

if basin_path.exists():
    basin_files = list(basin_path.glob('*.shp'))
    print(f"\n✓ Created basin shapefiles: {len(basin_files)}")
    
if network_path.exists():
    network_files = list(network_path.glob('*.shp'))
    print(f"✓ Created river network shapefiles: {len(network_files)}")
    
    # Load and check number of basins
    if basin_files:
        gdf = gpd.read_file(basin_files[0])
        print(f"\nNumber of sub-basins (GRUs): {len(gdf)}")
        print(f"Total area: {gdf.geometry.area.sum() / 1e6:.2f} km²")

## 7. Visualize the Distributed Domain

In [ ]:
# Visualize the delineated domain with stream network
basin_files = list((project_dir / 'shapefiles' / 'river_basins').glob('*.shp'))
network_files = list((project_dir / 'shapefiles' / 'river_network').glob('*.shp'))
    
if basin_files and network_files:
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Load data
    basins = gpd.read_file(basin_files[0])
    rivers = gpd.read_file(network_files[0])
    
    # Plot basins with different colors
    basins.plot(ax=ax, column='GRU_ID', cmap='viridis', 
               alpha=0.7, edgecolor='black', linewidth=0.5)
    
    # Plot river network
    rivers.plot(ax=ax, color='blue', linewidth=2)
    
    # Add pour point
    pour_point = gpd.read_file(pour_point_path)
    pour_point.plot(ax=ax, color='red', markersize=150, marker='o', zorder=5)
    
    ax.set_title(f'Distributed Domain: {len(basins)} Sub-basins', fontsize=16, fontweight='bold')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    
    # Add colorbar for GRU IDs
    sm = plt.cm.ScalarMappable(cmap='viridis', 
                               norm=plt.Normalize(vmin=basins['GRU_ID'].min(), 
                                                 vmax=basins['GRU_ID'].max()))
    sm._A = []
    cbar = fig.colorbar(sm, ax=ax, shrink=0.8)
    cbar.set_label('GRU ID', fontsize=12)
    
    plt.tight_layout()
    plt.show()

## 8. Geospatial Domain Definition - Discretization

Now we'll create Hydrologic Response Units (HRUs) based on the Grouped Response Units (GRUs) we just created.

In [ ]:
# Step 4: Geospatial Domain Definition - Discretization
print("=== Step 4: Geospatial Domain Definition - Discretization ===")

# Discretize domain
print(f"Creating HRUs based on GRUs...")
print(f"Method: {confluence.config['DOMAIN_DISCRETIZATION']}")
print("For this tutorial: 1 GRU = 1 HRU (simplest case)")

hru_path = confluence.managers['domain'].discretize_domain()

# Check the created HRU shapefile
catchment_path = project_dir / 'shapefiles' / 'catchment'
if catchment_path.exists():
    hru_files = list(catchment_path.glob('*.shp'))
    print(f"\n✓ Created HRU shapefiles: {len(hru_files)}")
    
    if hru_files:
        hru_gdf = gpd.read_file(hru_files[0])
        print(f"\nHRU Statistics:")
        print(f"Number of HRUs: {len(hru_gdf)}")
        print(f"Number of GRUs: {hru_gdf['GRU_ID'].nunique()}")
        print(f"Total area: {hru_gdf.geometry.area.sum() / 1e6:.2f} km²")
        
        # Show HRU distribution
        hru_counts = hru_gdf.groupby('GRU_ID').size()
        print(f"\nHRUs per GRU:")
        for gru_id, count in hru_counts.items():
            print(f"  GRU {gru_id}: {count} HRUs")

## 9. Model Agnostic Data Processing - Observed Data

The observed streamflow data will be the same for both the lumped and distributed models since they use the same pour point.

In [ ]:
# Step 5: Model Agnostic Data Processing - Observed Data
print("=== Step 5: Model Agnostic Data Processing - Observed Data ===")

# Check if we can reuse data from the lumped model
lumped_obs_path = CONFLUENCE_DATA_DIR / 'domain_Bow_at_Banff_lumped' / 'observations' / 'streamflow' / 'preprocessed'
can_reuse_obs = lumped_obs_path.exists() and list(lumped_obs_path.glob('*.csv'))

if can_reuse_obs:
    print("Found existing observed data from lumped model. Reusing...")
    # We can proceed, but CONFLUENCE will handle the reuse internally

# Process observed data
print("Processing observed streamflow data...")
confluence.managers['data'].process_observed_data()

# Visualize observed streamflow data
obs_path = project_dir / 'observations' / 'streamflow' / 'preprocessed' / f"{confluence.config['DOMAIN_NAME']}_streamflow_processed.csv"
if obs_path.exists():
    obs_df = pd.read_csv(obs_path)
    obs_df['datetime'] = pd.to_datetime(obs_df['datetime'])
    
    fig, ax = plt.subplots(figsize=(14, 6))
    ax.plot(obs_df['datetime'], obs_df['discharge_cms'], 
            linewidth=1.5, color='blue', alpha=0.7)
    
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Discharge (m³/s)', fontsize=12)
    ax.set_title(f'Observed Streamflow - Bow River at Banff (WSC Station: {confluence.config["STATION_ID"]})', 
                fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    
    # Add statistics
    ax.text(0.02, 0.95, f'Mean: {obs_df["discharge_cms"].mean():.1f} m³/s\nMax: {obs_df["discharge_cms"].max():.1f} m³/s', 
            transform=ax.transAxes, 
            bbox=dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.8),
            verticalalignment='top')
    
    plt.tight_layout()
    plt.show()

## 10. Model Agnostic Data Processing - Forcing Data

For the distributed model, we need forcing data for each GRU.

In [ ]:
# Step 6: Model Agnostic Data Processing - Forcing Data
print("=== Step 6: Model Agnostic Data Processing - Forcing Data ===")

# Check if we can reuse some forcing data from the lumped model
lumped_forcing_path = CONFLUENCE_DATA_DIR / 'domain_Bow_at_Banff_lumped' / 'forcing'
can_reuse_forcing = lumped_forcing_path.exists()

if can_reuse_forcing:
    print("Found existing forcing data from lumped model.")
    print("Note: Distributed model requires unique forcing for each GRU, so data will be reprocessed.")

# Acquire forcings
print(f"\nAcquiring forcing data: {confluence.config['FORCING_DATASET']}")
confluence.managers['data'].acquire_forcings()

## 11. Model Agnostic Data Processing - Preprocessing

In [ ]:
# Step 7: Model Agnostic Data Processing - Preprocessing
print("=== Step 7: Model Agnostic Data Processing - Preprocessing ===")

# Run model-agnostic preprocessing
print("\nRunning model-agnostic preprocessing...")
confluence.managers['data'].run_model_agnostic_preprocessing()

print("\n✓ Model-agnostic preprocessing completed")

## 12. Model-Specific Processing - Preprocessing

Now we prepare inputs specific to our chosen hydrological model (SUMMA in this case), set up for a distributed configuration.

In [ ]:
# Step 8: Model Specific Processing and Initialization
print("=== Step 8: Model Specific Processing and Initialization ===")

# Preprocess models
print(f"Preparing {confluence.config['HYDROLOGICAL_MODEL']} input files...")
print(f"Note: For distributed mode with {confluence.config['HYDROLOGICAL_MODEL']}, this includes generating:")
print(f"  - Model parameter files for each GRU")
print(f"  - Routing configuration for river network")
print(f"  - Connections between GRUs based on stream topology")

confluence.managers['model'].preprocess_models()

print("\n✓ Model-specific preprocessing completed")

## 13. Run the Distributed Model

Now we execute the SUMMA model in distributed mode with routing.

In [ ]:
# Step 9: Run the Distributed Model
print("=== Step 9: Run the Distributed Model ===")

# Run the model
print(f"Running distributed {confluence.config['HYDROLOGICAL_MODEL']} model...")
print(f"Number of GRUs: (check from previous output)")
print("Note: This will take longer than the lumped model due to multiple units.")

confluence.managers['model'].run_models()

print("\n✓ Model execution completed")

## 14. Distributed Model Benchmarking

Let's run benchmarking to evaluate the performance of our distributed model.

In [ ]:
# Step 10: Distributed Model Benchmarking
print("=== Step 10: Distributed Model Benchmarking ===")

# Run benchmarking
print("\nRunning benchmarking analysis...")
benchmark_results = confluence.managers['analysis'].run_benchmarking()

print("\n✓ Benchmarking completed")

## 15. Visualize Distributed Model Results

In [ ]:
# Postprocess results and visualize
print("Post-processing model results...")
confluence.managers['model'].postprocess_results()

# Load and plot simulation results
sim_path = project_dir / 'simulations' / confluence.config['EXPERIMENT_ID'] / 'mizuRoute'
sim_files = list(sim_path.glob('*.nc'))

if sim_files:
    # Load simulation data
    sim_data = xr.open_dataset(sim_files[0])
    
    # Load observation data
    obs_path = project_dir / 'observations' / 'streamflow' / 'preprocessed' / f"{confluence.config['DOMAIN_NAME']}_streamflow_processed.csv"
    obs_df = pd.read_csv(obs_path)
    obs_df['datetime'] = pd.to_datetime(obs_df['datetime'])
    
    # Find the segment ID for the outlet
    if 'reachID' in sim_data.dims:
        outlet_idx = sim_data.reachID.values == confluence.config['SIM_REACH_ID']
        if any(outlet_idx):
            # Extract simulated flow at outlet
            sim_flow = sim_data.IRFroutedRunoff.sel(reachID=confluence.config['SIM_REACH_ID']).to_pandas()
            sim_df = pd.DataFrame({'datetime': sim_flow.index, 'flow': sim_flow.values})
            
            # Plot comparison
            fig, ax = plt.subplots(figsize=(14, 6))
            
            # Plot observed flow
            ax.plot(obs_df['datetime'], obs_df['discharge_cms'], 
                    color='blue', linewidth=1.5, label='Observed')
            
            # Plot simulated flow
            ax.plot(sim_df['datetime'], sim_df['flow'], 
                    color='red', linewidth=1.5, alpha=0.7, label='Simulated (Distributed)')
            
            ax.set_xlabel('Date', fontsize=12)
            ax.set_ylabel('Discharge (m³/s)', fontsize=12)
            ax.set_title('Distributed Model Results - Bow River at Banff', 
                        fontsize=14, fontweight='bold')
            ax.grid(True, alpha=0.3)
            ax.legend(fontsize=10)
            
            plt.tight_layout()
            plt.show()
            
    else:
        print("Could not find reachID dimension in simulation output")
else:
    print("No simulation results found. Check model execution.")

## 16. Compare Lumped vs Distributed Results (Optional)

If you've completed the lumped model tutorial, we can compare results between the two approaches.

In [ ]:
# Try to load lumped model results for comparison
lumped_sim_path = CONFLUENCE_DATA_DIR / 'domain_Bow_at_Banff_lumped' / 'simulations' / 'run_1' / 'mizuRoute'
lumped_sim_files = list(lumped_sim_path.glob('*.nc')) if lumped_sim_path.exists() else []

if lumped_sim_files:
    print("Found lumped model results. Creating comparison plot...")
    
    # Load lumped simulation data
    lumped_data = xr.open_dataset(lumped_sim_files[0])
    
    # Load distributed simulation data
    dist_sim_path = project_dir / 'simulations' / confluence.config['EXPERIMENT_ID'] / 'mizuRoute'
    dist_sim_files = list(dist_sim_path.glob('*.nc'))
    
    if dist_sim_files:
        dist_data = xr.open_dataset(dist_sim_files[0])
        
        # Load observation data
        obs_path = project_dir / 'observations' / 'streamflow' / 'preprocessed' / f"{confluence.config['DOMAIN_NAME']}_streamflow_processed.csv"
        obs_df = pd.read_csv(obs_path)
        obs_df['datetime'] = pd.to_datetime(obs_df['datetime'])
        
        # Extract flows
        # Find the segment ID for the outlet
        if 'reachID' in dist_data.dims and 'reachID' in lumped_data.dims:
            # Extract distributed flow
            dist_flow = dist_data.IRFroutedRunoff.sel(reachID=confluence.config['SIM_REACH_ID']).to_pandas()
            dist_df = pd.DataFrame({'datetime': dist_flow.index, 'flow': dist_flow.values})
            
            # Extract lumped flow
            lumped_flow = lumped_data.IRFroutedRunoff.sel(reachID=confluence.config['SIM_REACH_ID']).to_pandas()
            lumped_df = pd.DataFrame({'datetime': lumped_flow.index, 'flow': lumped_flow.values})
            
            # Plot comparison
            fig, ax = plt.subplots(figsize=(14, 6))
            
            # Plot observed flow
            ax.plot(obs_df['datetime'], obs_df['discharge_cms'], 
                    color='black', linewidth=1.5, label='Observed')
            
            # Plot lumped flow
            ax.plot(lumped_df['datetime'], lumped_df['flow'], 
                    color='blue', linewidth=1.5, alpha=0.7, label='Lumped Model')
            
            # Plot distributed flow
            ax.plot(dist_df['datetime'], dist_df['flow'], 
                    color='red', linewidth=1.5, alpha=0.7, label='Distributed Model')
            
            ax.set_xlabel('Date', fontsize=12)
            ax.set_ylabel('Discharge (m³/s)', fontsize=12)
            ax.set_title('Lumped vs Distributed Model Comparison - Bow River at Banff', 
                        fontsize=14, fontweight='bold')
            ax.grid(True, alpha=0.3)
            ax.legend(fontsize=10)
            
            plt.tight_layout()
            plt.show()
            
    else:
        print("Distributed simulation results not found. Run the distributed model first.")
else:
    print("Lumped model results not found. Complete Tutorial 2 first for comparison.")

## 17. Optional Steps - Optimization and Analysis

These steps are optional and could be run to further explore the distributed model.

In [ ]:
# Step 11: Optional Steps (Optimization and Analysis)
print("=== Step 11: Optional Steps ===")

# Check if optimization is enabled
if confluence.config.get('RUN_ITERATIVE_OPTIMISATION', False):
    print("Running model calibration...")
    optimization_results = confluence.managers['optimization'].calibrate_model()
else:
    print("Model calibration skipped (RUN_ITERATIVE_OPTIMISATION = False)")

# Check if emulation is enabled
if confluence.config.get('RUN_LARGE_SAMPLE_EMULATION', False) or confluence.config.get('RUN_RANDOM_FOREST_EMULATION', False):
    print("Running parameter emulation...")
    emulation_results = confluence.managers['optimization'].run_emulation()
else:
    print("Parameter emulation skipped")

# Check if decision analysis is enabled
if confluence.config.get('RUN_DECISION_ANALYSIS', False):
    print("Running decision analysis...")
    decision_results = confluence.managers['analysis'].run_decision_analysis()
else:
    print("Decision analysis skipped")

# Check if sensitivity analysis is enabled
if confluence.config.get('RUN_SENSITIVITY_ANALYSIS', False):
    print("Running sensitivity analysis...")
    sensitivity_results = confluence.managers['analysis'].run_sensitivity_analysis()
else:
    print("Sensitivity analysis skipped")

## 18. Compare Domain Structures: Lumped vs Distributed

Let's create a visual comparison of the domain structures.

In [ ]:
# Create comparison visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Lumped model visualization (if exists)
lumped_basin_path = CONFLUENCE_DATA_DIR / 'domain_Bow_at_Banff_lumped' / 'shapefiles' / 'river_basins'
if lumped_basin_path.exists():
    lumped_files = list(lumped_basin_path.glob('*.shp'))
    if lumped_files:
        lumped_basin = gpd.read_file(lumped_files[0])
        lumped_basin.plot(ax=ax1, color='lightblue', edgecolor='navy', linewidth=2)
        ax1.set_title('Lumped Model\n(1 Unit)', fontsize=14, fontweight='bold')
        ax1.axis('off')

# Distributed model visualization
dist_basin_path = project_dir / 'shapefiles' / 'river_basins'
dist_network_path = project_dir / 'shapefiles' / 'river_network'

if dist_basin_path.exists() and dist_network_path.exists():
    dist_basin_files = list(dist_basin_path.glob('*.shp'))
    dist_network_files = list(dist_network_path.glob('*.shp'))
    
    if dist_basin_files and dist_network_files:
        basins = gpd.read_file(dist_basin_files[0])
        network = gpd.read_file(dist_network_files[0])
        
        basins.plot(ax=ax2, column='GRU_ID', cmap='viridis', 
                   edgecolor='black', linewidth=0.5, alpha=0.7)
        network.plot(ax=ax2, color='blue', linewidth=2)
        ax2.set_title(f'Distributed Model\n({len(basins)} GRUs)', fontsize=14, fontweight='bold')
        ax2.axis('off')

plt.suptitle('Lumped vs Distributed Domain Structure', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 19. Summary and Key Differences

### What we accomplished:
1. Created a distributed model with multiple GRUs
2. Used stream network delineation
3. Maintained 1:1 relationship between GRUs and HRUs
4. Ran the same model (SUMMA) in distributed mode
5. Reused data from the lumped model where possible

### Key differences from lumped model:
- **Multiple spatial units**: Several GRUs instead of one
- **River routing**: mizuRoute connects the GRUs
- **More detailed representation**: Captures spatial variability
- **Longer computation time**: More units to simulate

### Next steps:
1. Compare performance metrics between lumped and distributed
2. Experiment with different stream thresholds
3. Try different discretization methods (elevation bands, land cover)
4. Calibrate parameters for individual GRUs

In [ ]:
# Print final summary
print("=== Distributed Workflow Complete ===\n")
print(f"Project: {confluence.config['DOMAIN_NAME']}")
print(f"Method: {confluence.config['DOMAIN_DEFINITION_METHOD']}")
print(f"Stream Threshold: {confluence.config['STREAM_THRESHOLD']}")
print(f"Model: {confluence.config['HYDROLOGICAL_MODEL']}")
print(f"\nKey outputs:")
print(f"  - Basin shapefile: {project_dir}/shapefiles/river_basins/")
print(f"  - River network: {project_dir}/shapefiles/river_network/")
print(f"  - Model results: {project_dir}/simulations/{confluence.config['EXPERIMENT_ID']}/")
print(f"  - Comparison plots: {project_dir}/plots/results/")

# Get workflow status
status = confluence.workflow_orchestrator.get_workflow_status()
print(f"\nWorkflow Status:")
print(f"Total steps: {status['total_steps']}")
print(f"Completed steps: {status['completed_steps']}")
print(f"Pending steps: {status['pending_steps']}")

## 20. Alternative - Run Complete Workflow

Instead of running the workflow step by step, you could use CONFLUENCE's built-in workflow orchestration to run the entire process at once.

In [ ]:
# Alternative: Run the complete workflow in one step
# (Uncomment to use this instead of the step-by-step approach)

# confluence.run_workflow()